## Dataset refresh

This code assumes there is a workspace named {const.tenantRoot}1, and that there is a deployed datamodel to that workspace with a valid connection.   It discovers the workspace and dataset ids and then initiatiates a refresh.

In [ ]:
pip install azure-mgmt-sql requests msal pyodbc requests_toolbelt

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
from services.aadservice import AadService
credential = AadService.get_credential()

## Part 1 - Find Next Tenant Name


In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
scope = 'https://analysis.windows.net/powerbi/api/.default'
aadPBIToken = credential.get_token(scope).token

pbiApiHeaders =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadPBIToken}

In [ ]:
#retreive the list of workspaces
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups'       

apiResponse = requests.get(apiUrl, headers=pbiApiHeaders)
#error handling for API call
if apiResponse.status_code != 200:
    description = f'Error retreiving workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

In [ ]:
from services.env import const

workspaces = apiResponse["value"]

workspaceId = "blank"
keyValue = 1

for workspace in workspaces:
    if workspace["name"] == f"{const.tenantRoot}{keyValue}":
        workspaceId = workspace["id"]

print(f"The workspace id is: {workspaceId}")

In [ ]:
#/myorg/groups/{workspace_id}/datasets
#This retreives a list of datasets in the given workspace, notice the response value is wrapped in an array []
import requests
import json


apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets'       

apiResponse = requests.get(apiUrl, headers=pbiApiHeaders)
if apiResponse.status_code != 200:
    description = f'Error while retrieving datasets:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
    #print(api_response.status_code, description=description)
else:
    apiResponse = json.loads(apiResponse.text)
    datasets = apiResponse["value"]
    dataset = datasets[0]
    datasetId = dataset["id"]

    print(f"Deployed model has datasetId: {datasetId}")

In [ ]:
#This initiates the dataset refresh for the given workspace and dataset.
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/refreshes'       

body = {
  "notifyOption": "NoNotification",
  "retryCount": 3
}

apiResponse = requests.post(apiUrl, data=json.dumps(body), headers=pbiApiHeaders)
if apiResponse.status_code != 202:
    description = f'Error while retrieving datasets:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
    #print(api_response.status_code, description=description)
else:
    print(f"Refresh on dataset {datasetId} in workspace {workspaceId} initiated Successfully")